In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import log_loss
from sklearn.cross_validation import KFold
import re
from sklearn.linear_model import LogisticRegression

/opt/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from mochi import *

In [3]:
import datetime

In [4]:
#lodaing data
data_path = "/home/raku/kaggleData/2sigma/xgb142/"
train_file = data_path + "xgb1.42-train.json"
test_file = data_path + "xgb1.42-test.json"
train_df = pd.read_json(train_file)
test_df = pd.read_json(test_file)
print(train_df.shape)
print(test_df.shape)


(49352, 293)
(74659, 292)


In [5]:
numericals = [u'bath_per_bed',u'bathrooms',u'bedrooms',u'building0',u'cluster_id_10_d',u'cluster_id_30_d',u'dayofyear',
 u'latitude',u'listing_id',u'longitude',u'm14perf',u'm14perf_f',u'm30perf',u'm30perf_f',u'm3perf',u'm3perf_f',
 u'm7perf',u'm7perf_f',u'm_c_distance',u'm_m_distance',u'manager_id_nrank',u'manager_id_perf',u'mlat',
 u'mlon',u'num_description_words',u'num_features', u'num_photos',
 u'price',u'price_per_bath',u'price_per_bed',u'price_per_room',]

numerical_may_processed = [ u'created_day',u'created_hour',u'created_month',u'time_stamp'] 

In [6]:
new_hcc_categoricals=[ u'building_id',u'cluster_id_10',u'cluster_id_30',u'street_name']
hcc_categoricals = ['manager_id','house_type']

In [9]:
#statiscals
statistical = []
for feature in test_df.columns:
    if re.match('((manager_id)|(house_type))\S+((mean)|(median)|(min)|(max))',feature) !=None:
        statistical.append(feature)

In [11]:
#features
with_feat = []
for feature in test_df.columns:
    if re.match('with_\S+',feature) !=None:
        statistical.append(feature)

In [12]:
new_hcc_feature = []

skf=KFold(len(train_df['interest_level']),5,shuffle=True,random_state = 42)
#hcc encoding for the old hcc features
for feature in hcc_categoricals:    
    for train,test in skf:
        hcc_scoring(train_df.iloc[train,:],train_df.iloc[test,:],feature,'high',\
                   update_df = train_df)
        hcc_scoring(train_df.iloc[train,:],train_df.iloc[test,:],feature,'medium',\
                   update_df = train_df)

    hcc_scoring(train_df,test_df,feature,'high')
    hcc_scoring(train_df,test_df,feature,'medium')
    new_hcc_feature.append('hcc_'+feature+'_high')
    new_hcc_feature.append('hcc_'+feature+'_medium')

transferred=['another_day','another_hour']
train_df["created"] = pd.to_datetime(train_df["created"],unit='ms')
test_df["created"] = pd.to_datetime(test_df["created"],unit='ms')

train_df['another_day']=(train_df['created']+pd.tseries.offsets.DateOffset(days=15)).dt.day
train_df['another_hour']=(train_df['created']+pd.tseries.offsets.DateOffset(hours=12)).dt.hour
test_df['another_day']=(test_df['created']+pd.tseries.offsets.DateOffset(days=15)).dt.day
test_df['another_hour']=(test_df['created']+pd.tseries.offsets.DateOffset(hours=12)).dt.hour

train_df['pic_created']=train_df['time_stamp'].apply(datetime.datetime.fromtimestamp)
test_df['pic_created']=test_df['time_stamp'].apply(datetime.datetime.fromtimestamp)

train_df["pic_month"] = train_df["pic_created"].dt.month
test_df["pic_month"] = test_df["pic_created"].dt.month
train_df["pic_day"] = train_df["pic_created"].dt.day
test_df["pic_day"] = test_df["pic_created"].dt.day
train_df["pic_hour"] = train_df["pic_created"].dt.hour
test_df["pic_hour"] = test_df["pic_created"].dt.hour

train_df['another_pic_day']=(train_df['pic_created']+pd.tseries.offsets.DateOffset(days=15)).dt.day
train_df['another_pic_hour']=(train_df['pic_created']+pd.tseries.offsets.DateOffset(hours=12)).dt.hour
test_df['another_pic_day']=(test_df['pic_created']+pd.tseries.offsets.DateOffset(days=15)).dt.day
test_df['another_pic_hour']=(test_df['pic_created']+pd.tseries.offsets.DateOffset(hours=12)).dt.hour

transferred.extend(['pic_month','pic_day','pic_hour','another_pic_day','another_pic_hour'])

In [13]:
processing_features = numericals+numerical_may_processed+new_hcc_feature+transferred+statistical

In [14]:
normalized_train = train_df.copy()
normalized_test = test_df.copy()

train_test = pd.concat([train_df.drop('interest_level',axis=1),test_df])

for f in processing_features:
    normalized_train[f]=normalized_train[f].fillna(train_test[f].median())
    normalized_test[f]=normalized_test[f].fillna(train_test[f].median())
        
for f in processing_features:
    normalized_train[f]=(normalized_train[f]-train_test[f].mean())/train_test[f].std()
    normalized_test[f]=(normalized_test[f]-train_test[f].mean())/train_test[f].std()

#store the basic transformed train and test
#normalized_train.to_json(data_path+'normal_train_df.json')
#normalized_test.to_json(data_path+'normal_test_df.json')

In [15]:
#prepare for training
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=KFold(len(train_df),5,shuffle=True,random_state = 2333)

In [16]:
features=list(processing_features)+with_feat

In [17]:
#first edition:
#numericals from xgb142 + some new hcc encoding + with_feat from xgb142
cv_scores=[]
for dev_index, val_index in KF:
    dev_set, val_set = normalized_train.iloc[dev_index,:] , normalized_train.iloc[val_index,:] 
    dev_X, val_X = dev_set[features].as_matrix(), val_set[features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
            #random forest us
    lr = LogisticRegression()
    lr.fit(dev_X,dev_y)
    preds = lr.predict_proba(val_X)
        
    cv_scores.append(log_loss(val_y, preds))
        

    print(cv_scores)
    break
print np.mean(cv_scores)


[0.58050456098964887]
0.58050456099


In [24]:
new_new_hcc_features=[]
#new cat_gen_features
skf=KFold(len(train_df['interest_level']),5,shuffle=True,random_state = 42)
#hcc encoding for the old hcc features
for feature in new_hcc_categoricals:
    for train,test in skf:
        hcc_scoring(train_df.iloc[train,:],train_df.iloc[test,:],feature,'high',\
                   update_df = train_df)
        hcc_scoring(train_df.iloc[train,:],train_df.iloc[test,:],feature,'medium',\
                   update_df = train_df)
        performance_eval(train_df.iloc[train,:],train_df.iloc[test,:],feature,\
                   update_df = train_df,random=0.01)

    hcc_scoring(train_df,test_df,feature,'high')
    hcc_scoring(train_df,test_df,feature,'medium')
    performance_eval(train_df,test_df,feature,random=0.01)
    new_new_hcc_features.append('hcc_'+feature+'_high')
    new_new_hcc_features.append('hcc_'+feature+'_medium')
    new_new_hcc_features.append(feature+'_nrank_s_r')
    new_new_hcc_features.append(feature+'_perf_s_r')


In [25]:
normalized_train = train_df.copy()
normalized_test = test_df.copy()

train_test = pd.concat([train_df.drop('interest_level',axis=1),test_df])

for f in new_new_hcc_features:
    normalized_train[f]=normalized_train[f].fillna(train_test[f].median())
    normalized_test[f]=normalized_test[f].fillna(train_test[f].median())
        
for f in new_new_hcc_features:
    normalized_train[f]=(normalized_train[f]-train_test[f].mean())/train_test[f].std()
    normalized_test[f]=(normalized_test[f]-train_test[f].mean())/train_test[f].std()

In [26]:
features=list(processing_features)+with_feat+new_new_hcc_features

In [27]:
#first edition:
#numericals from xgb142 + some new hcc encoding + with_feat from xgb142
cv_scores=[]
for dev_index, val_index in KF:
    dev_set, val_set = normalized_train.iloc[dev_index,:] , normalized_train.iloc[val_index,:] 
    dev_X, val_X = dev_set[features].as_matrix(), val_set[features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
            #random forest us
    lr = LogisticRegression()
    lr.fit(dev_X,dev_y)
    preds = lr.predict_proba(val_X)
        
    cv_scores.append(log_loss(val_y, preds))
        

    print(cv_scores)
    break
print np.mean(cv_scores)


[0.77667739242905409]
0.776677392429
